In [ ]:
!pip install tensorflow

In [ ]:
!pip install PIL

In [ ]:
!pip install scipy

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [7]:

# Parameter
IMG_SIZE = (224, 224)  # Ukuran gambar untuk MobileNetV2
BATCH_SIZE = 32
EPOCHS = 30
LEARNING_RATE = 0.001
FINE_TUNE_EPOCHS = 20  # Tambahan epoch untuk fine-tuning
FINE_TUNE_LR = 1e-5    # Learning rate untuk fine-tuning

# Direktori dataset
train_dir = "train_batik"
test_dir = "test_batik_2"

# Augmentasi Data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
test_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)


# Generator Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# Informasi jumlah kelas
num_classes = len(train_generator.class_indices)

Found 928 images belonging to 29 classes.


Found 232 images belonging to 29 classes.


In [ ]:


# Model Transfer Learning
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Bekukan layer pada model dasar untuk transfer learning

# Tambahkan lapisan khusus
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout untuk mencegah overfitting
x = Dense(256, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)

# Buat model
model = Model(inputs=base_model.input, outputs=predictions)

# Kompilasi model untuk transfer learning
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Callback
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6)


In [ ]:

# Pelatihan awal (transfer learning)
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=[reduce_lr]
)



Epoch 1/30


C:\Users\ACER\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


29/29 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.0756 - loss: 3.7083 - val_accuracy: 0.3276 - val_loss: 2.7148 - learning_rate: 0.0010
Epoch 2/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 71s 2s/step - accuracy: 0.3365 - loss: 2.4270 - val_accuracy: 0.4483 - val_loss: 2.1401 - learning_rate: 0.0010
Epoch 3/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - accuracy: 0.4979 - loss: 1.8227 - val_accuracy: 0.5172 - val_loss: 1.9345 - learning_rate: 0.0010
Epoch 4/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 0.5208 - loss: 1.6210 - val_accuracy: 0.5259 - val_loss: 1.7620 - learning_rate: 0.0010
Epoch 5/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 0.6045 - loss: 1.3919 - val_accuracy: 0.5560 - val_loss: 1.7453 - learning_rate: 0.0010
Epoch 6/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.6242 - loss: 1.2588 - val_accuracy: 0.5517 - val_loss: 1.5803 - learning_rate: 0.0010
Epoch 7/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.6739 - loss: 1.1108 - val_accuracy: 0.5603 - va

In [11]:
# Fine-tuning
# Buka beberapa lapisan terakhir dari base model
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Bekukan semua kecuali 20 lapisan terakhir
    layer.trainable = False

# Kompilasi ulang model dengan learning rate yang lebih kecil
model.compile(optimizer=Adam(learning_rate=FINE_TUNE_LR),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Fine-tuning model
history_fine_tune = model.fit(
    train_generator,
    epochs=FINE_TUNE_EPOCHS,
    validation_data=test_generator,
    callbacks=[reduce_lr]
)

# Evaluasi model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Epoch 1/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 108s 3s/step - accuracy: 0.5783 - loss: 1.4744 - val_accuracy: 0.6207 - val_loss: 1.4451 - learning_rate: 1.0000e-05
Epoch 2/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.5956 - loss: 1.3123 - val_accuracy: 0.6293 - val_loss: 1.4816 - learning_rate: 1.0000e-05
Epoch 3/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.6228 - loss: 1.2372 - val_accuracy: 0.6121 - val_loss: 1.4382 - learning_rate: 1.0000e-05
Epoch 4/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - accuracy: 0.6652 - loss: 1.0671 - val_accuracy: 0.6121 - val_loss: 1.3832 - learning_rate: 1.0000e-05
Epoch 5/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 95s 3s/step - accuracy: 0.6646 - loss: 1.1192 - val_accuracy: 0.6293 - val_loss: 1.5183 - learning_rate: 1.0000e-05
Epoch 6/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 106s 3s/step - accuracy: 0.6552 - loss: 1.1010 - val_accuracy: 0.6293 - val_loss: 1.5244 - learning_rate: 1.0000e-05
Epoch 7/20
29/29 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.6657 - los

In [7]:
model.save("model_10_20_epochs.h5")  # Menyimpan model ke file my_model.h5